# Using the geojson generator

## Import the packages

In [1]:
import pandas as pd

In [2]:
from convert2geojson import Convert2GeoJson
from citableclass import Citable

## Load data from citable

The data can be provided in various ways. This tutorial uses the `citable`format of Edition Topoi, see [here](http://www.edition-topoi.org/publishing_with_us/citable) for details (currently in German only). 

In [3]:
cite = Citable('10.17171/1-1')

For data from Edition Topoi the format of data can be checked in metadata `cite.metadata()`. In this case, the excel data can be downloaded with `cite.excel()`, which saves the data locally as `temp.xlsx`, which is returned by the method. We can then create a dataframe by providing the path to the data.

In [4]:
out = cite.collection()

In [5]:
res = []
for key, val in out.items():
    try:
        res.append((val['doi'], val['holder_id'], val['metadata']['General Information']['Subtitle'], val['holder']['latitude'], val['holder']['longitude']))
    except KeyError:
        pass


In [6]:
df = pd.DataFrame(res, columns=['doi', 'holder_id', 'subtitle','latitude', 'longitude']).dropna()

In [7]:
df.head(2)

,doi,holder_id,subtitle,latitude,longitude
0,10.17171/1-1-1,ItalyAltino,"Altino, Inv. Nr. AL10",45.5463,12.3990
3,10.17171/1-1-34,GreeceKomotini,"Komotini, Inv. Nr. 1358",41.1262,25.4186


## Build GeoJSON and save to disc

Note that not every dataframe row contains information on longitude or latitude. These rows are dropped from the dataframe. Therefore, the number of found features may vary.

Instantiate using the dataframe, a list of properties to be displayed in popups, and the column titles for the geodata. By adding `.convert()` the geojson is created directly after instantiation.

In [8]:
build = Convert2GeoJson(dataframe=df, properties=df.columns, lat='latitude',lon='longitude').convert()

You can provide additional information for the geoJSON by adding an attribution (`attribution='Generated by ...'`) and a description (`description='Shows the ...`).


The generated geoJSON in dictionary format can be accessed by `.geojson()`.

In [9]:
geojson = build.geojson()
geojson.keys()

dict_keys(['type', 'features', 'properties'])

To save the generated geoJSON as a file, chose

In [10]:
#build.save('./data/testGEO.geojson')

You can also provide parameters for the filename (`name='data.geojson'`) or path (`path='.'`).

## Display result

The package uses ipyleaflet to generate the map for the geoJSON data. 

Markers on maps can be drawn fully (standard) or grouped with leaflets `MarkerCluster` (style='grouped'). The basemap for the background mapping is loaded from CartoDBs 'base-light-nolabels' source. Additonal parameters include customized basemaps or layouting for the map.

In [11]:
mapD = build.display(style='grouped')

In [12]:
mapD

Map(center=[36.3976985464396, 15.306339349845205], controls=(ZoomControl(options=['position', 'zoom_in_text', …